In [ ]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import os

from helper_functions import elementlines, nearestlines, theoretical_energy

%matplotlib qt

In [ ]:
hs.eds.electron_range('Cu', 10, density='auto', tilt=0)
# hs.eds.electron_range('Cu', 30, density='auto', tilt=0)
# hs.eds.electron_range('Cu', 30, density='auto', tilt=0)

In [ ]:
##### SEM Apreo March 2023 #####
path = '../../Masteroppgave/2023-03-08_EDS-Apreo/exports/'
aztec_file = '../../Masteroppgave/2023-03-08_EDS-Apreo/AZtec k-factors and quantification.xlsx'
df = pd.read_excel(aztec_file, sheet_name=4)

df['AZ at%'] = df['AZ at%'] / 100

In [ ]:
df.head(3)

In [ ]:
def add_name_x_kV_pA(df, add_name=False):
    # add a name column to the dataframe
    df['Name'] = df['File']
    df['Name'] = df['Name'].str.replace('_', ' ')
    # df['Name'] = df['Name'].str.replace('GaAs', '(a)')
    # df['Name'] = df['Name'].str.replace('GaSb', '(b)')
    df['Name'] = df['Name'].str.replace('processTime', 'PT')
    df['Name'] = df['Name'].str.replace('scratched', 'scr')
    df['i'] = 0.0
    df['i/1000'] = 0.0
    df['i/i_sum'] = 0.0

    # add kV to the dataframe
    kv_list = df['Name'].str.split(' ') 
    pA_list = df['Name'].str.split(' ') 
    for j in range(len(kv_list)):
        kv_list[j] = int(kv_list[j][1][:-2])
        pA_list[j] = int(pA_list[j][2][:-2])
    df['kV'] = kv_list
    df['pA'] = pA_list

    # print(df['Name'].str.split(' '))
    df['Name'] = df['Name'].str.replace(' ', '<br>') # for plotting
    df['Name'] = df['Group'] + '<br>' + df['Name']



    # add a x-axis for plotting
    # df['x'] = 0
    # x = 0
    # for f in df['File'].unique():
        # df.loc[df['File'] == f, 'x'] = x
        # x += 1

    return df

df = add_name_x_kV_pA(df)

In [ ]:
# while working now, I do not need "i corr" "k-ratio"
df = df.drop(columns=['i corr', 'k-ratio'])

In [ ]:
df.head(3)

In [ ]:
def calculate_intensities(filename):

        # check if filename exists in path
    if filename + '.emsa' not in os.listdir(path):
        print(filename + '.emsa', 'not found in path')
        return
    
    df_s = df[df['File'] == filename]
    df_s = df_s.sort_values(by=['Element']) # alphabetically, as HS

    noise_slice = 0.65 # keV

    # make the spectrum and the model
    s = hs.load(path + filename + '.emsa', signal_type='EDS_TEM')
    elements = df_s['Element'].values
    s = s.isig[noise_slice:] # noise peak slice
    if s.metadata.Acquisition_instrument.TEM.beam_energy < 20:
        s = s.isig[:s.metadata.Acquisition_instrument.TEM.beam_energy]
    s.add_elements(elements)
    s.add_lines()
    m = s.create_model()
    m.fit()
    m.fit_background()

    lines = df_s['Line'].values

    line_intensities = []
    for l in lines:
        line_intensities.append(round(m[l].A.value))

    relative_intensities = []  # this is wt%, I think (according to Castaing's paper)
    for line_intens in line_intensities:
        relative_intensities.append(round(line_intens/sum(line_intensities), 3))
    
    df_s['i'] = line_intensities
    df_s['i/1000'] = [round(i/1000, 3) for i in line_intensities]

    df_s['i/i_sum'] = relative_intensities

    df[df['File'] == filename] = df_s

# calculate_intensities('GaAs_30kV_50pA')

In [ ]:
for filename in df['File'].unique():
    calculate_intensities(filename)

at_wt_list = []
# as the iteration is over all the rows, some operations are repeated, but it is not a big deal.
for i in range(len(df)):
    at_wt = hs.material.elements[df['Element'][i]].General_properties.atomic_weight
    at_wt_list.append(at_wt)
df['at_wt'] = at_wt_list

# i/i_sum is wt%

### at%_1 = (wt%_1 / at_wt_1) / (wt%_1 / at_wt_1 + wt%_2 / at_wt_2)

In [ ]:
elements_list = []
at_perc_list =[]
for xi in df['x'].unique():
    elements_list.append(df[df['x']==xi]['Element'].unique().tolist())
    elements_list.append(df[df['x']==xi]['Element'].unique().tolist()) # twice as there are two elements per sample

    wt = df[df['x']==xi]['i/i_sum'].tolist()
    at_wt = df[df['x']==xi]['at_wt'].tolist()

    # at%_1 = (wt%_1 / at_wt_1) / (wt%_1 / at_wt_1 + wt%_2 / at_wt_2)
    # https://www.plasmaterials.com/converting-atomic-percent-to-weight-percent-and-vice-versa/
    at_perc_list.append((wt[0]/at_wt[0]) / (wt[0]/at_wt[0] + wt[1]/at_wt[1]))
    at_perc_list.append((wt[1]/at_wt[1]) / (wt[0]/at_wt[0] + wt[1]/at_wt[1]))

df['Elements'] = elements_list
df['i: at%'] = at_perc_list
df.head(1)

In [ ]:
df[['File', 'AZ at%', 'Line', 'i', 'i/i_sum','i: at%']][::2]

# Initial quantification

AZtec and "raw" wt\% as (i1/(i1+i2)), calculated into at\%

In [ ]:
# ### Initial quantification

# df2 = df[['File',  'Line', 'kV', 'x', 'i',  'AZ at%','i: at%']]
# df2['i'] = df2['i'] / 1000 
# df2['AZ at%'] = df2['AZ at%'] * 100
# df2['i: at%'] = df2['i: at%'] * 100
# df2

# ##

# # selected_files = ['GaAs_05kV_25pA', 'GaAs_30kV_50pA', 'GaSb_15kV_50pA', 'GaSb_30kV_50pA', 'GaSb_30kV_50pA_processTime1']
# # # df2 = df[df['File'].isin(selected_files)]
# # df2[['File', 'kV', 'AZ at%', 'Line', 'i', 'i: at%']]
# df2

In [ ]:
# plotly plot

# fig = px.bar(df, x='Name', y='i/i_sum', color='Line', barmode='group', title='Relative intensities of lines with no correction')
# fig.update_xaxes(tickangle=90)
# fig.show()

In [ ]:
# plotting stuff
x = df['x'].values
y = df['i: at%'].values
y2 = df['AZ at%'].values
name = df['Name'].values
line = df['Line'].values
element = df['Element'].values

lines_list_plotting = ['Sb_La', 'As_Ka', 'Ga_Ka', 'Ga_La', 'As_La']
elements_list_plotting = ['Ga', 'As', 'Sb']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'] 
# df['Name'] = df['Name'].str.replace('kV', ' kV')
# df['Name'] = df['Name'].str.replace('pA', ' pA')
# df['Name'] = df['Name'].str.replace('PT', 'PT ')

In [ ]:
fig = go.Figure()
legends = []
# looping over all the spectra in the df
for i in range(len(x)):
    col = colors[lines_list_plotting.index(line[i])]
    col = colors[elements_list_plotting.index(element[i])]
    show_leg = False if element[i] in legends else True
    fig.add_scatter(x=[x[i]], y=[y[i]], mode='markers', name=f'{element[i]},  area ratio', marker_color=col, legendgroup=element[i], 
                    showlegend=show_leg, marker_size=10)
    fig.add_scatter(x=[x[i]], y=[y2[i]], mode='markers', name=f'{element[i]},  AZtec', marker_color=col, marker_symbol='x', 
                    legendgroup=element[i], showlegend=show_leg, marker_size=10)
    legends.append(f'{element[i]}')

fig.update_xaxes(tickvals=df['x'].unique(), ticktext=df['Name'].unique(), range=[-0.5, len(df['x'].unique())-0.5])
fig.update_yaxes(range=[0.33, 0.67])
fig.update_layout(yaxis_title='Composition [at%]', height=450, width=1100,
                  legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.995, orientation="h",),
                  margin=dict(l=5, r=5, t=5, b=5, pad=0),
                  font=dict(family="EB Garamond SemiBold", size=16, color="black"))

# fig.write_image('figures/initial_quantification.pdf')
# fig.write_image('../mastersthesis/figures/results/initial_quantification.pdf')
fig

# ZAF absorption corrections

With approximated depth.

$I / I_0 =  exp(-\mu * (\rho*t))$,

- where $\mu$ (or mu/rho) is the mass absorption coefficient (from HS)
- $\rho$ is the density of the element (from HS, as the two densities added together and divided by 2)
- t is the travel distance, estimated as the xray_range divided by 2, 3 and 4 (also from HS, based on Anderson and Hasler (1966), K-O)
- I_0 is the generated intensity,
- I is the measured intensity


$I_0 = I / exp(-\mu * (\rho*t))$

## What the code do

- First some functions to calculate the $\mu$ and $\rho$ values from the HS data
- Then a function to calculate the path length, t, from the xray_range
    - This value is divided by 1, 2, 3 and 4 to get the different t values as approximations for the average PL
- Then a function to calculate the I_0 values from the I and t values

- The absorption correction factors are calculated for each line in the spectrum, using the different t
- Then the absorption is corrected for each line in the spectrum, using the different t

In [ ]:
# hs.material.mass_absorption_mixture(elements=elements, weight_percent=weight_percent, energies=line)
# gives the mass absorption coefficient of the line in cm^2/g

def get_density_mixture(elements, weight_percent=None, atomic_percent=None):
    """
    Returns the density for a mixture of elements.

    Parameters
    ----------
    elements : list
        List of elements in the mixture.
    weight_percent : list, optional
        The weight percent of each element in the mixture. The default is [50,50].

    Returns
    -------
    float
        The density for the mixture.
    """
    if weight_percent is None:
        if atomic_percent is None:
            raise ValueError('Either weight_percent or atomic_percent must be given.')
        else:
            # hs.material.atomic_to_weight([50, 50], ['Ga', 'Sb'])
            weight_percent = hs.material.atomic_to_weight(atomic_percent, elements)
    d1 = hs.material.elements[elements[0]].Physical_properties.density_gcm3
    d2 = hs.material.elements[elements[1]].Physical_properties.density_gcm3
    return (d1*weight_percent[0] + d2*weight_percent[1])/100


def electron_range_KO(elements, line, e0):
    # the Kanaya-Okayama parameterziation, Goldstein Eq. 22.5
    # r = (0.0276 * A)/(Z^0.92*rho)*(E_0^1.67-E_C^1.67)
    # where A is the atomic mass, Z is the atomic number, rho is the density, E_0 is the beam energy, and E_C is the critical energy
    # In this approximation, the average A, Z, and rho is used for the mixture
    avg_A = sum([hs.material.elements[elements[0]].General_properties.atomic_weight,
                hs.material.elements[elements[1]].General_properties.atomic_weight])/2
    avg_Z = sum([hs.material.elements[elements[0]].General_properties.Z,
                hs.material.elements[elements[1]].General_properties.Z])/2 
    avg_rho = sum([hs.material.elements[elements[0]].Physical_properties.density_gcm3,
                   hs.material.elements[elements[1]].Physical_properties.density_gcm3])/2
    e_c = theoretical_energy(line)
    r = (0.0276 * avg_A)/(avg_Z**0.92*avg_rho)*(e0**1.67-e_c**1.67)
    return r

def get_path_length(elements:list, beam_energy: float, *, line: str, range_divider: float = 2, 
                    angle: float = 35, weight_percent: list = None, atomic_percent: list = None ) -> float:
    """
    Crude assumnption that the path length (PL, t, Psi) is the electron range divided by range_divider.
    Kinda gets the average path length as the center of the interaction volume, but is a crude assumption.
    Tested with range_divider = 1, 2, 3, 4

    Parameters
    ----------
    elements : list
        List of elements in the mixture. Use the first element
    line : str
        The line to use for the calculation.
    beam_energy : float
        The beam energy in keV.
    weight_percent : list, optional
        The weight percent of each element in the mixture. The default is [50,50].
    angle : float, optional
        The take-off angle of the detector in degrees. The default is 35.

    Returns
    -------
    float
        The travel distance for the mixture, in cm.
    """
    um_to_cm = 1e-4
    if weight_percent is None:
        if atomic_percent is None:
            raise ValueError('Either weight_percent or atomic_percent must be given.')
        else:
            # hs.material.atomic_to_weight([50, 50], ['Ga', 'Sb'])
            weight_percent = hs.material.atomic_to_weight(atomic_percent, elements)
    rho = get_density_mixture(elements, weight_percent=weight_percent)
    # electron_range = hs.eds.electron_range(elements[0], beam_energy, density=rho, tilt=0) # in um


    r_max = electron_range_KO(elements, line, beam_energy) # in um

    r = r_max  / range_divider 
    # calculate PL = electron_range * csc(angle)
    # where csc(angle) = 1/sin(angle), and angle is in degrees
    path_length = r / np.sin(np.deg2rad(angle))
    return path_length  * um_to_cm



def calculate_i_gen(i, elements, line, beam_energy, weight_percent=None, atomic_percent=None, r_div=2):
    """
    Returns the generated intensity for a mixture of elements.

    Using i = 1 will give the absorption correction factor, 
    which later can be multiplied with the measured intensity to get the generated intensity.

    i0 = i / exp(-mu * rho * t)

    Parameters
    ----------
    i : float
        The measured intensity.
    elements : list
        List of elements in the mixture.
    line : str
        The line to use for the calculation.
    beam_energy : float
        The beam energy in keV.
    weight_percent : list, optional
        The weight percent of each element in the mixture. The default is [50,50].

    Returns
    -------
    float
        The generated intensity for the mixture.
    """
    if weight_percent is None:
        if atomic_percent is None:
            raise ValueError('Either weight_percent or atomic_percent must be given.')
        else:
            # hs.material.atomic_to_weight([50, 50], ['Ga', 'Sb'])
            weight_percent = hs.material.atomic_to_weight(atomic_percent, elements)
    mu = hs.material.mass_absorption_mixture(elements=elements, weight_percent=weight_percent, energies=line)
    rho = get_density_mixture(elements, weight_percent=weight_percent)
    t = get_path_length(elements, beam_energy, line=line, weight_percent=weight_percent, range_divider=r_div)
    return i / np.exp(-mu * rho * t)

In [ ]:
# for line in ['Ga_La',  'Sb_La']: # 'Ga_Ka',
#     print(f'\n{line}: {theoretical_energy(line):.2f} keV, in {elems}')
#     elems =['Ga', 'Sb']
#     for e0 in [30, 15, 10, 5]:
#         # print max electron range
#         print(f'{e0} keV, t: {electron_range(elems, line=line, e0=e0):.2f} um')


In [ ]:
# for elems in [['Ga', 'As'], ['Ga', 'Sb']]:
#         # print rho
#         print(f'{elems}, rho: {get_density_mixture(elems, atomic_percent=[50,50]):.2f} g/cm$^3$')

In [ ]:
# calculate mu, rho, t, i_0, and A (i=1), and add to df

# mu_list = []
rho_list = []
# t_list = []
# i0_r1_list = []
r1_A_list = []
r2_A_list = []
r3_A_list = []
r4_A_list = []

for j in range(len(df)):
    # mu_list.append(round(get_mass_absorption_coeff_mixture(df['Elements'][j], df['Line'][j]), 1))
    rho_list.append(get_density_mixture(df['Elements'][j], atomic_percent=[50, 50]))
    r1_A_list.append(calculate_i_gen(i=1, elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j], r_div=1, atomic_percent=[50, 50]))
    r2_A_list.append(calculate_i_gen(i=1, elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j], r_div=2, atomic_percent=[50, 50]))
    r3_A_list.append(calculate_i_gen(i=1, elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j], r_div=3, atomic_percent=[50, 50]))
    r4_A_list.append(calculate_i_gen(i=1, elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j], r_div=4, atomic_percent=[50, 50]))
    # t_list.append(get_path_length(df['Elements'][j], df['kV'][j]))
    # i0_r1_list.append(round(calculate_i_gen(i=df['i'][j], elements=df['Elements'][j], line=df['Line'][j], beam_energy=df['kV'][j], r_div=1), 1))

# df['r1_A'] = r1_A_list
df['r2_A'] = r2_A_list
df['r3_A'] = r3_A_list
df['r4_A'] = r4_A_list

# df['mu'] = mu_list
# df['rho'] = rho_list
# df['t'] = t_list
# df['i0_r1'] = i0_r1_list

df[['Group', 'kV', 'Line', 'i', 'AZ at%', 'i: at%',  'r2_A', 'r3_A', 'r4_A']].head(3) # 'r1_A',

In [ ]:
df[['File', 'Group', 'kV', 'Line', 'i', 'r2_A', 'r3_A', 'r4_A']]

In [ ]:
# calculate the atomic percent with the absorption correction factor

def use_absorption_correction(absorption_correction_column_name: str, df=df):
    a_col = absorption_correction_column_name
    at_corrected_list = []
    # each xi is one spectrum
    for xi in df['x'].unique():
        xi_elements = df[df['x']==xi]['Elements'].values[0]
        i = df[df['x']==xi]['i'].values
        a_corr = df[df['x']==xi][a_col].values
        i0 = i * a_corr  # correct the intensity
        wt_corrected = [i0[0]/i0.sum(), i0[1]/i0.sum()]
        at_corrected = hs.material.weight_to_atomic(wt_corrected, xi_elements)
        # print(at_corrected)
        at_corrected_list += at_corrected.tolist()
    # print(at_corrected_list)
    df[f'{a_col}_at%'] = at_corrected_list


In [ ]:
use_absorption_correction('r4_A')
use_absorption_correction('r3_A')
use_absorption_correction('r2_A')

In [ ]:
# df['i: at%'] = df['i: at%'] * 100

In [ ]:

df[['Group','Line',  'kV', 'pA', 'i/1000', 'i: at%', 'r2_A_at%', 'r3_A_at%', 'r4_A_at%']]

In [ ]:
# calculate i0/i0_sum for each x, and add to df

def calculate_relative_intensity(column: str, df=df):
    """
    Returns the relative intensity for each x value in the dataframe.
    """
    relative_intensities = []
    for j in range(len(df)):
        i0 = df[column][j]
        i0_sum = df[df['x']==df['x'][j]][column].sum()
        # print(i0/i0_sum)
        relative_intensities.append(round(i0/i0_sum, 3))
    return relative_intensities

i0_r1_sum_list = calculate_relative_intensity('i0_r1')
i0_r2_sum_list = calculate_relative_intensity('i0_r2')
i0_r3_sum_list = calculate_relative_intensity('i0_r3')
i0_r4_sum_list = calculate_relative_intensity('i0_r4')

df['i0_r1/sum'] = i0_r1_sum_list
df['i0_r2/sum'] = i0_r2_sum_list
df['i0_r3/sum'] = i0_r3_sum_list
df['i0_r4/sum'] = i0_r4_sum_list
df.head(3)


In [ ]:
def calculate_atom_percent(column: str, df=df):
    """
    Returns the atom percent from the weight percent for each x value in the dataframe.
    """
    atom_percent_list = []
    for xi in df['x'].unique():
        wt_i0 = df[df['x']==xi][column].tolist()
        at_wt = df[df['x']==xi]['at_wt'].tolist()
        # at%_1 = (wt%_1 * at_wt_1) / (wt%_1 * at_wt_1 + wt%_2 * at_wt_2)
        atom_percent_list.append((wt_i0[0]/at_wt[0]) / (wt_i0[0]/at_wt[0] + wt_i0[1]/at_wt[1]))
        atom_percent_list.append((wt_i0[1]/at_wt[1]) / (wt_i0[0]/at_wt[0] + wt_i0[1]/at_wt[1]))
    return atom_percent_list

df['Elements'] = elements_list
# df['i0: at%'] = calculate_atom_percent('i0/i0_sum')
df['r1 at%'] = calculate_atom_percent('i0_r1/sum')
df['r2 at%'] = calculate_atom_percent('i0_r2/sum')
df['r3 at%'] = calculate_atom_percent('i0_r3/sum')
df['r4 at%'] = calculate_atom_percent('i0_r4/sum')


df.head(3)

In [ ]:
df[['File', 'Line', 'AZ at%', 'i/i_sum', 'i0/i0_sum','i0: at%', 'i: at%', 'at_wt']]

In [ ]:
fig = go.Figure()
legends = []
for i in range(len(x)):
    # col = colors[lines_list_plotting.index(line[i])]
    col = colors[elements_list_plotting.index(df['Element'][i])]
    showlegend = False if line[i] in legends else True
    marker_line_width = 1 if ("La" in line[i]) else 0
    fig.add_scatter(x=[x[i]], y=[df['i/i_sum'][i]], mode='markers', name=f'{line[i]}, i wt%', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, symbol='circle', line=dict(color='Black', width=marker_line_width)))
    fig.add_scatter(x=[x[i]], y=[y2[i]], mode='markers', name=f'{line[i]}, AZ at%', marker_symbol='x', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, line=dict(color='Black', width=marker_line_width)))
    fig.add_scatter(x=[x[i]], y=[df['i0/i0_sum'][i]], mode='markers', name=f'{line[i]}, i0 wt%', marker_symbol='diamond', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, line=dict(color='Black', width=marker_line_width)))
    legends.append(f'{line[i]}')

# x ticks are the filenames
fig.update_xaxes(tickvals=df['x'].unique(), ticktext=df['Name'].unique(), tickangle=90, range=[-0.5, len(df['x'].unique())-0.5])
# fig.update_yaxes(range=[0.33, 0.67])
fig.update_layout(title='AZtec at% and wt% for raw intensity (i, circle) and A-corrected intensity (i0, diamonds)', yaxis_title='Relative intensity')
fig.update_layout(height=600, width=1200)
fig.update_layout(legend=dict(x=1.01, y=0.5, traceorder='normal', font=dict(family='sans-serif', size=12, color='black'))) 
fig.show()

In [ ]:
fig = go.Figure()
legends = []
for i in range(len(x)):
    # col = colors[lines_list_plotting.index(line[i])]
    col = colors[elements_list_plotting.index(df['Element'][i])]
    showlegend = False if line[i] in legends else True
    marker_line_width = 1 if ("La" in line[i]) else 0
    fig.add_scatter(x=[x[i]], y=[df['i: at%'][i]], mode='markers', name=f'{line[i]}, i at%', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, symbol='circle', line=dict(color='Black', width=marker_line_width)))
    fig.add_scatter(x=[x[i]], y=[y2[i]], mode='markers', name=f'{line[i]}, AZ at%', marker_symbol='x', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, line=dict(color='Black', width=marker_line_width)))
    fig.add_scatter(x=[x[i]], y=[df['i0: at%'][i]], mode='markers', name=f'{line[i]}, i0 at%', marker_symbol='diamond', legendgroup=line[i], showlegend=showlegend,
                    marker=dict(size=10, color=col, line=dict(color='Black', width=marker_line_width)))
    legends.append(f'{line[i]}')

# x ticks are the filenames
fig.update_xaxes(tickvals=df['x'].unique(), ticktext=df['Name'].unique(), tickangle=90, range=[-0.5, len(df['x'].unique())-0.5])
# fig.update_yaxes(range=[0.33, 0.67])
fig.update_layout(title='AZtec at% and at% for i (circle) and i0 (diamonds)', yaxis_title='Relative intensity')
fig.update_layout(height=600, width=1200)
fig.update_layout(legend=dict(x=1.01, y=0.5, traceorder='normal', font=dict(family='sans-serif', size=12, color='black'))) 
fig.show()

In [ ]:
# write a output file
df.to_csv('output.csv', index=False)

In [ ]:
df

In [ ]:
stop here to see the plot below!

In [ ]:
fig = go.Figure()

y1 = df['i/i_sum']
# y2 = df['i_gen/sum']
y2t2 = df['i_gen_t2/sum']
y2t3 = df['i_gen_t3/sum']
y2t4 = df['i_gen_t4/sum']

y3 = df['AZ at%']

fig.add_trace(go.Scatter(x=x, y=y1, name='Measured Intensity', mode='markers'))
# fig.add_trace(go.Scatter(x=x, y=y2, name='A corrected Intensity', mode='markers'))
fig.add_trace(go.Scatter(x=x, y=y2t2, name='A corrected Intensity, t2', mode='markers'))
fig.add_trace(go.Scatter(x=x, y=y2t3, name='A corrected Intensity, t3', mode='markers'))
fig.add_trace(go.Scatter(x=x, y=y2t4, name='A corrected Intensity, t4', mode='markers'))

fig.add_trace(go.Scatter(x=x, y=y3, name='AZ comp', mode='markers'))

fig.update_xaxes(tickvals=df['x'].unique(), ticktext=df['Name'].unique(), tickangle=90, range=[-0.5, len(df['x'].unique())-0.5])


fig


fig.update_layout(title='Intensity with and without A correction', yaxis_title='Relative intensity')